# Chargement des données

In [25]:
# code inspiré de https://git.regardscitoyens.org/regardscitoyens/sprint-scrutins/blob/master/votes_an/ancsv.py

import glob, json, csv

# recupere les députés

députés = list(csv.DictReader(open('liste_deputes_libre_office.csv')))
députés_par_id = {int(député["identifiant"]): député for député in députés}

# recuperer les votes

votes_par_scrutins = {}

for fichier in glob.glob('scrutins/*.json'):
    scrutin = json.load(open(fichier))['scrutin']

    id_scrutin = scrutin['numero']
    votes_par_scrutins[id_scrutin] = {}

    if int(scrutin['syntheseVote']['nombreVotants']) < 100:
        continue
        
    groupes = scrutin['ventilationVotes']['organe']['groupes']['groupe']

    for groupe in groupes:
        dn = groupe['vote']['decompteNominatif']
        
        for position in ('pour', 'contre', 'abstention', 'nonVotant'):
            votants_position = dn['%ss' % position]
            
            if not votants_position:
                continue
            
            votants = votants_position['votant']

            if not isinstance(votants, list):
                votants = [votants]

            for votant in votants:
                votes_par_scrutins[id_scrutin][int(votant['acteurRef'][2:])] = position



def similarité(député1, député2):
    global votes
    total = 0
    total_en_commun = 0
    for scrutin, vote in votes_par_scrutins.items():
        if not vote.get(député1) or not vote.get(député2):
            continue
        if vote.get(député1) == vote.get(député2):
            total_en_commun += 1
        total += 1
    if total == 0:
        return 0
    return 1 - total_en_commun / total

print('sim mel/ruffin:', similarité(2150, 722142))
print('sim bergé/ruffin:', similarité(722046, 722142))

sim mel/ruffin: 0.017699115044247815
sim bergé/ruffin: 0.9553571428571429


# Carte

In [26]:
import umap
import umap.plot

def metric(a, b):
    return similarité(int(a[0]), int(b[0]))

députés_id = [int(x) for x in list(sorted(députés_par_id.keys()))]
partis = [députés_par_id[id]['Groupe politique (complet)'] for id in députés_id]
noms = [(députés_par_id[id]['Nom'] + ' ' + députés_par_id[id]['Prénom']) for id in députés_id]

mapper = umap.UMAP(metric=metric, random_state=30).fit([[id] for id in députés_id])

umap.plot.output_notebook()
p = umap.plot.interactive(mapper, labels=partis, hover_data={"parti": partis, "nom": noms})
umap.plot.show(p)

/home/damien/repos/umap/umap/umap_.py:1495: UserWarning: custom distance metric does not return gradient; inverse_transform will be unavailable. To enable using inverse_transform method method, define a distance function that returns a tuple of (distance [float], gradient [np.array])
  "custom distance metric does not return gradient; inverse_transform will be unavailable. "


Loading BokehJS ...

# Députés les plus similaires à François Ruffin

In [23]:
sims = [[id, similarité(id, 722142)] for id in députés_id]
sims.sort(key=lambda x: -x[1])
for sim in sims:
    print((députés_par_id[sim[0]]['Nom'] + ' ' + députés_par_id[sim[0]]['Prénom']), sim[1])

Coquerel Éric 1.0
Corbière Alexis 1.0
Ruffin François 1.0
Quatennens Adrien 0.9929577464788732
Lachaud Bastien 0.9922480620155039
Taurine Bénédicte 0.9914529914529915
Autain Clémentine 0.9910714285714286
Rubin Sabine 0.9910714285714286
Ressiguier Muriel 0.9908256880733946
Fiat Caroline 0.9858156028368794
Obono Danièle 0.9848484848484849
Panot Mathilde 0.984251968503937
Prud'homme Loïc 0.9830508474576272
Bernalicis Ugo 0.9826086956521739
Mélenchon Jean-Luc 0.9823008849557522
Ratenon Jean-Hugues 0.9758064516129032
Larive Michel 0.975609756097561
Jumel Sébastien 0.9440559440559441
Dharréville Pierre 0.9426751592356688
Chassaigne André 0.9347826086956522
Wulfranc Hubert 0.9323308270676691
Bruneel Alain 0.9318181818181818
Peu Stéphane 0.9312977099236641
Lecoq Jean-Paul 0.9302325581395349
Buffet Marie-George 0.9253731343283582
Faucillon Elsa 0.9247311827956989
Bello Huguette 0.922077922077922
Dufrègne Jean-Paul 0.9054054054054054
Roussel Fabien 0.8952380952380953
Brotherson Moetai 0.86111111